<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий на hh.ru
   

In [237]:
from IPython.display import display
import pandas as pd
import re
import plotly.express as px
import plotly.graph_objs as pg
import numpy as np
import plotly.io as pio
pio.renderers.default = "svg"

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [238]:
df = pd.read_csv('data/dst-3.0_16_1_hh_database.csv', sep=';')

2. Выведите несколько первых (последних строк таблицы), чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [239]:
df.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано


3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице

4. Обратите внимание на информацию о числе непустых значений

In [240]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Пол, возраст                     44744 non-null  object
 1   ЗП                               44744 non-null  object
 2   Ищет работу на должность:        44744 non-null  object
 3   Город, переезд, командировки     44744 non-null  object
 4   Занятость                        44744 non-null  object
 5   График                           44744 non-null  object
 6   Опыт работы                      44576 non-null  object
 7   Последнее/нынешнее место работы  44743 non-null  object
 8   Последняя/нынешняя должность     44742 non-null  object
 9   Образование и ВУЗ                44744 non-null  object
 10  Обновление резюме                44744 non-null  object
 11  Авто                             44744 non-null  object
dtypes: object(12)
memory usage: 4.1+

5. Выведите основную статистическую информацию о столбцах.


In [241]:
df_desc1 = df.describe()
df_desc = {}
for col in df.columns:
    top_freq = df[col].value_counts(normalize=True).max()
    nunique_ratio = df[col].nunique() / df[col].count()
    df_desc[col] = {
        'Кол-во': df_desc1[col]['count'],
        'Уникальных значений': df_desc1[col]['unique'],
        'Уникальных значений, %': f'{round(nunique_ratio*100, 2)}',
        'Одинаковых значений, %': f'{round(top_freq*100, 2)}',
        'Самое частое значение': df_desc1[col]['top'],
        'Кол-во самого частого значения': df_desc1[col]['freq']
    }

display(pd.DataFrame(df_desc).transpose())

,Кол-во,Уникальных значений,"Уникальных значений, %","Одинаковых значений, %",Самое частое значение,Кол-во самого частого значения
"Пол, возраст",44744,16003,35.77,0.04,"Мужчина , 32 года , родился 17 сентября 1986",18
ЗП,44744,690,1.54,9.08,50000 руб.,4064
Ищет работу на должность:,44744,14929,33.37,6.93,Системный администратор,3099
"Город, переезд, командировки",44744,10063,22.49,2.82,"Москва , не готов к переезду , не готов к кома...",1261
Занятость,44744,38,0.08,67.11,полная занятость,30026
График,44744,47,0.11,50.79,полный день,22727
Опыт работы,44576,44413,99.63,0.01,Опыт работы 10 лет 8 месяцев Апрель 2018 — по...,3
Последнее/нынешнее место работы,44743,30214,67.53,2.09,Индивидуальное предпринимательство / частная п...,935
Последняя/нынешняя должность,44742,16927,37.83,4.61,Системный администратор,2062
Образование и ВУЗ,44744,40148,89.73,0.01,Высшее образование 1987 Военный инженерный Кра...,4


# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [242]:
def education_transformer(v: str) -> str:
    for level in ['высшее', 'неоконченное высшее', 'среднее специальное', 'среднее']:
        if v.lower().startswith(level):
            return level
    return np.nan

df['Образование'] = df['Образование и ВУЗ'].apply(func=education_transformer)
# display(df[df['Образование'].isnull()].head()) # Проверить все ли категории нашли
print(f"Cредний уровень образования имеют {df[df['Образование'] == 'среднее'].shape[0]} соискателей." )
df.drop('Образование и ВУЗ', axis=1, inplace=True)
display(df.head())

Cредний уровень образования имеют 559 соискателей.


,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,неоконченное высшее
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,высшее
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,среднее специальное
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,высшее
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,высшее


2. Теперь нас интересует столбец **"Пол/возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [243]:
template = '([?М|Ж])(.*) , (.*) (.*) , (.*)'
df['Пол'] = df['Пол, возраст'].apply(func=lambda x: re.search(template, x).group(1))
df['Возраст'] = df['Пол, возраст'].apply(func=lambda x: int(re.search(template, x).group(3)))
# display(df[['Пол', 'Bозраст']])
# display(df[df['Возраст'].isnull()])
print(f"{round(df['Пол'].value_counts(normalize=True)['Ж'] * 100, 2)} процентов женских резюме представлено в наших данных.")
print(f"{round(df['Возраст'].mean(), 1)} - средний возраст соискателей.")
df.drop('Пол, возраст', axis=1, inplace=True)
display(df.head())

19.07 процентов женских резюме представлено в наших данных.
32.2 - средний возраст соискателей.


,ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование,Пол,Возраст
0,29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,неоконченное высшее,М,39
1,40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,высшее,М,60
2,20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,среднее специальное,Ж,36
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,высшее,М,38
4,140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,высшее,Ж,26


3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-третьих, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [244]:
def experience_transformer(v):
    if pd.isnull(v):
        return np.nan
    variant1 = re.search(r'^Опыт работы (\d+) (год|года|лет) (\d+) (месяцев|месяца|месяц) {2}(.*)', v)
    if variant1:
        return int(variant1.group(1)) * 12 + int(variant1.group(3))
    variant2 = re.search(r'^Опыт работы (\d+) (месяцев|месяца|месяц) {2}(.*)$', v)
    if variant2:
        return int(variant2.group(1))
    variant3 = re.search(r'^Опыт работы (\d+) (год|года|лет) {2}(.*)$', v)
    if variant3:
        return int(variant3.group(1)) * 12
    if v == 'Не указано':
        return 0
    return -1

df['Опыт работы (месяц)'] = df['Опыт работы'].apply(func=experience_transformer)
# display(df[df['Опыт работы (месяц)'] == -1])
print(f"Медианный опыт работы (в месяцах) равен {round(df['Опыт работы (месяц)'].median())}")
df.drop('Опыт работы', axis=1, inplace=True)
display(df)

Медианный опыт работы (в месяцах) равен 100


,ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование,Пол,Возраст,Опыт работы (месяц)
0,29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...","МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,неоконченное высшее,М,39,202.0
1,40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,высшее,М,60,233.0
2,20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,среднее специальное,Ж,36,123.0
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,высшее,М,38,225.0
4,140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,высшее,Ж,26,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44739,50000 руб.,"Финансист, аналитик, экономист, бухгалтер, мен...","Тверь , готов к переезду (Москва, Химки) , гот...",полная занятость,"полный день, удаленная работа","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,22.04.2019 12:32,Не указано,высшее,М,30,91.0
44740,39000 руб.,"Системный администратор, IT-специалист","Липецк , готов к переезду , готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день, ...",ИП Пестрецов,Предприниматель,22.04.2019 13:11,Не указано,высшее,М,27,84.0
44741,40000 руб.,"Аналитик данных, Математик","Челябинск , готова к переезду , готова к редки...",полная занятость,"полный день, удаленная работа","ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,09.04.2019 05:07,Не указано,высшее,Ж,48,257.0
44742,20000 руб.,Контент-менеджер,"Тамбов , не готов к переезду , не готов к кома...","частичная занятость, полная занятость",удаленная работа,IQ-Maxima,Менеджер проектов,26.04.2019 14:25,Имеется собственный автомобиль,высшее,М,24,46.0


4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов миллионников:
    
   <code> million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    
    
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [245]:
million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград' ]

def city_transformer(v):
    city = v.split(" , ")[0]
    if city in million_cities:
        return 'город миллионник'
    if city in ['Москва', 'Санкт-Петербург']:
        return city
    return 'другие'

def relocation_transformer(v):
    tokens = v.split(" , ")
    value = tokens[1] if ' перее' in tokens[1] else tokens[2]
    return 'не ' not in value.split('(')[0]

def business_trip_transformer(v):
    search = ' командиров'
    tokens = v.split(" , ")
    if len(tokens) > 2 and search in tokens[2]:
        value = tokens[2]
    elif len(tokens) > 3 and search in tokens[3]:
        value = tokens[3]
    else:
        return True
    return 'не ' not in value


df['Город'] = df['Город, переезд, командировки'].apply(func=city_transformer)
df['Готовность к переезду'] = df['Город, переезд, командировки'].apply(func=relocation_transformer)
df['Готовность к командировкам'] = df['Город, переезд, командировки'].apply(func=business_trip_transformer)
# display(df[df['Готовность к переезду'] == 'Санкт-Петербург'])
# display(df['Готовность к командировкам'].unique())
print(f"{round(df['Город'].value_counts(normalize=True)['Санкт-Петербург'] * 100)} процентов соискателей живут в Санкт-Петербурге")
print(f"{round(df[df['Готовность к переезду'] & df['Готовность к командировкам']].shape[0]/df.shape[0] * 100)} процентов соискателей готовы одновременно и к переездам, и к командировкам")
df.drop('Город, переезд, командировки', axis=1, inplace=True)
display(df)

11 процентов соискателей живут в Санкт-Петербурге
32 процентов соискателей готовы одновременно и к переездам, и к командировкам


,ЗП,Ищет работу на должность:,Занятость,График,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование,Пол,Возраст,Опыт работы (месяц),Город,Готовность к переезду,Готовность к командировкам
0,29000 руб.,Системный администратор,"частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...","МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,неоконченное высшее,М,39,202.0,другие,False,False
1,40000 руб.,Технический писатель,"частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,высшее,М,60,233.0,другие,False,True
2,20000 руб.,Оператор,полная занятость,полный день,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,среднее специальное,Ж,36,123.0,другие,False,False
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,высшее,М,38,225.0,другие,False,True
4,140000 руб.,Региональный менеджер по продажам,полная занятость,полный день,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,высшее,Ж,26,67.0,Москва,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44739,50000 руб.,"Финансист, аналитик, экономист, бухгалтер, мен...",полная занятость,"полный день, удаленная работа","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,22.04.2019 12:32,Не указано,высшее,М,30,91.0,другие,True,True
44740,39000 руб.,"Системный администратор, IT-специалист","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день, ...",ИП Пестрецов,Предприниматель,22.04.2019 13:11,Не указано,высшее,М,27,84.0,другие,True,True
44741,40000 руб.,"Аналитик данных, Математик",полная занятость,"полный день, удаленная работа","ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,09.04.2019 05:07,Не указано,высшее,Ж,48,257.0,город миллионник,True,True
44742,20000 руб.,Контент-менеджер,"частичная занятость, полная занятость",удаленная работа,IQ-Maxima,Менеджер проектов,26.04.2019 14:25,Имеется собственный автомобиль,высшее,М,24,46.0,другие,False,False


5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [246]:
mapping = {
    'Занятость': ['полная занятость', 'частичная занятость', 'проектная работа', 'волонтерство', 'стажировка'],
    'График': ['полный день', 'сменный график', 'гибкий график', 'удаленная работа', 'вахтовый метод']
}
for key in mapping:
    for column in mapping[key]:
        df[column] = df[key].apply(lambda x: column in x)

print(f"{df[df['проектная работа'] & df['волонтерство']].shape[0]} людей ищут проектную работу или волонтёрство")
print(f"{df[df['вахтовый метод'] & df['гибкий график']].shape[0]} людей хотят работать вахтовым методом или с гибким графиком")
# display(df['волонтерство'].unique())
df.drop(['Занятость', 'График'], axis=1, inplace=True)
display(df)

436 людей ищут проектную работу или волонтёрство
2311 людей хотят работать вахтовым методом или с гибким графиком


,ЗП,Ищет работу на должность:,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование,Пол,Возраст,Опыт работы (месяц),...,полная занятость,частичная занятость,проектная работа,волонтерство,стажировка,полный день,сменный график,гибкий график,удаленная работа,вахтовый метод
0,29000 руб.,Системный администратор,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,неоконченное высшее,М,39,202.0,...,True,True,True,False,False,True,True,True,True,True
1,40000 руб.,Технический писатель,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,высшее,М,60,233.0,...,True,True,True,False,False,True,True,True,True,False
2,20000 руб.,Оператор,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,среднее специальное,Ж,36,123.0,...,True,False,False,False,False,True,False,False,False,False
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,высшее,М,38,225.0,...,True,True,True,False,False,False,False,True,True,False
4,140000 руб.,Региональный менеджер по продажам,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,высшее,Ж,26,67.0,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44739,50000 руб.,"Финансист, аналитик, экономист, бухгалтер, мен...","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,22.04.2019 12:32,Не указано,высшее,М,30,91.0,...,True,False,False,False,False,True,False,False,True,False
44740,39000 руб.,"Системный администратор, IT-специалист",ИП Пестрецов,Предприниматель,22.04.2019 13:11,Не указано,высшее,М,27,84.0,...,True,True,True,False,False,True,True,True,True,False
44741,40000 руб.,"Аналитик данных, Математик","ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,09.04.2019 05:07,Не указано,высшее,Ж,48,257.0,...,True,False,False,False,False,True,False,False,True,False
44742,20000 руб.,Контент-менеджер,IQ-Maxima,Менеджер проектов,26.04.2019 14:25,Имеется собственный автомобиль,высшее,М,24,46.0,...,True,True,False,False,False,False,False,False,True,False


6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [247]:
er = pd.read_csv('data/ExchangeRates.csv', sep=',')
er.drop(['per', 'time', 'vol'], axis=1, inplace=True)
er['date'] = pd.to_datetime(er['date']).dt.date
cur_mapping = {
    'руб.': 'RUB',
    'бел.руб.': 'BYN',
    'KZT': 'KZT',
    'EUR': 'EUR',
    'USD': 'USD',
    'грн.': 'UAH',
    'сум': 'UZS',
    'KGS': 'KGS',
    'AZN': 'AZN'
}

df['date'] = pd.to_datetime(df['Обновление резюме']).dt.date
df['salary'] = df['ЗП'].apply(lambda x: int(x.split(' ')[0].strip()))
df['currency'] = df['ЗП'].apply(lambda x: cur_mapping[x.split(' ')[1].strip()])

df = df.merge(er, how='left', left_on=['currency','date'], right_on=['currency', 'date'])
df = df.fillna(
    {
        'close': 1,
        'proportion': 1,
    }
)
df['ЗП (руб)'] = round(df['salary'] * df['close'] / df['proportion'], 2)

df.drop(['ЗП', 'salary', 'currency', 'date', 'close', 'proportion'], axis=1, inplace=True)
print(f"{round(df['ЗП (руб)'].median() / 1000)} - медианная заработная плата соискателей в нашей таблице (в рублях) округленная до тысяч.")

display(df)

59 - медианная заработная плата соискателей в нашей таблице (в рублях) округленная до тысяч.


,Ищет работу на должность:,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование,Пол,Возраст,Опыт работы (месяц),Город,...,частичная занятость,проектная работа,волонтерство,стажировка,полный день,сменный график,гибкий график,удаленная работа,вахтовый метод,ЗП (руб)
0,Системный администратор,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,неоконченное высшее,М,39,202.0,другие,...,True,True,False,False,True,True,True,True,True,29000.0
1,Технический писатель,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,высшее,М,60,233.0,другие,...,True,True,False,False,True,True,True,True,False,40000.0
2,Оператор,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,среднее специальное,Ж,36,123.0,другие,...,False,False,False,False,True,False,False,False,False,20000.0
3,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,высшее,М,38,225.0,другие,...,True,True,False,False,False,False,True,True,False,100000.0
4,Региональный менеджер по продажам,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,высшее,Ж,26,67.0,Москва,...,False,False,False,False,True,False,False,False,False,140000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44739,"Финансист, аналитик, экономист, бухгалтер, мен...","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,22.04.2019 12:32,Не указано,высшее,М,30,91.0,другие,...,False,False,False,False,True,False,False,True,False,50000.0
44740,"Системный администратор, IT-специалист",ИП Пестрецов,Предприниматель,22.04.2019 13:11,Не указано,высшее,М,27,84.0,другие,...,True,True,False,False,True,True,True,True,False,39000.0
44741,"Аналитик данных, Математик","ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,09.04.2019 05:07,Не указано,высшее,Ж,48,257.0,город миллионник,...,False,False,False,False,True,False,False,True,False,40000.0
44742,Контент-менеджер,IQ-Maxima,Менеджер проектов,26.04.2019 14:25,Имеется собственный автомобиль,высшее,М,24,46.0,другие,...,True,False,False,False,False,False,False,True,False,20000.0


# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [248]:
x = 'Возраст'
title = 'Распределение по возрасту'

fig = px.histogram(df, x=x, title=title)
fig.update_layout(yaxis_title="Количество")
fig.show()

fig = px.box(df, x=x, title=title, points="all")
fig.show()

display(df[(df[x] < 16) | (df[x] > 75)])

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


Распределение по возрасту похоже на нормальное с перекосом в правую сторону.
Мода распределения - 30 лет.
Предельные значения - от 14 до 100 лет. Большинство соискателей, судя по коробчатой диаграмме, имеют возраст примерно от 27 до 36 лет.

Заметна аномалия в районе 100 лет, запись №33654.
Такое очень маловероятно, учитывая что соискатель имеет опыт работы всего 30 месяцев.
C другого конца еще одна аномалия - 14-ти летний соискатель с 477 месяцами опыта, запись №29098.

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
x = 'Опыт работы (месяц)'
title = 'Распределение по опыту работы (в месяцах)'

fig = px.histogram(df, x=x, title=title)
fig.update_layout(yaxis_title="Количество")
fig.show()

fig = px.box(df, x=x, title=title, points="all")
fig.show()

print(f"Мода опыта работы (месяц) - {df[x].mode()}")
display(df[(df[x] > 622) | (df[x]/12 > df['Возраст'])])

Мода опыта работы (месяц) - 81.
Предельные значения от 0 до 1188 месяцев. Большинство кандидатов имеют опыт от 57 до 154 месяцев.
Заметна аномалия в 1198 месяцев, запись №40015.
Она наводит на мысль поиска остальных записей, где опыт работы превосходит возраст соискателя.

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
x = 'ЗП (руб)'
title = 'Распределение по опыту работы (в месяцах)'

adjusted_df = df[df[x] <= 1_000_000]

fig = px.histogram(adjusted_df, x=x, title=title)
fig.update_layout(yaxis_title="Количество")
fig.show()

fig = px.box(df, x=x, title=title, points="all")
fig.show()

print(f"Мода зарплаты (руб) - {df[x].mode()}")
display(df[(df[x] > 1_000_000) ])

Модальное значение желаемой зарплаты - 50 тысяч рублей.
Предельные значения от 1 до 24304876 рублей. Большинство значений желаемой зарплаты лежат в промежутке между 37 и 95 тысяч рублей.
Мы наблюдаем 5 соискателей с желаемой зарплатой больше 1 миллиона рублей.
Для показа диаграмм заказчику, нужно убирать крайние значения т.к. требуемые зарплаты похоже являются ошибкой ввода.
Причина может быть либо в дополнительном нуле введенным по ошибке или ошибочно введенной валюте зарплаты.

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
data = adjusted_df.groupby('Образование', as_index=False)['ЗП (руб)'].median()
fig = px.bar(data, x='Образование', y='ЗП (руб)', title='Зависимость медианной желаемой зарплаты от образования')
fig.show()

Наибольшее значение желаемой зарплаты мы видим у соискателей с высшим образованием и наименьшее у соискателей со средним.
По диаграмме видно, что желаемая зарплата почти линейно зависит уровня образования.
Что безусловно важно для прогнозирования заработной платы.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
data = adjusted_df.groupby('Город', as_index=False)['ЗП (руб)'].median()
fig = px.bar(data, x='Город', y='ЗП (руб)', title='Зависимость желаемой зарплаты от города')
fig.show()

# display(df[df['ЗП (руб)'] > 923_000])

Желаемая зарплата в Москве более чем в 2 раза больше, чем в прочих городах России за исключением Санкт-Петербурга.
В Санкт-Петербурге зарплата в 1.5 раза больше, чем других городах за исключением Москвы.
Очевидно что признак города очень важен для прогнозирования.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
data = df.copy()
data = data.groupby(['Готовность к переезду', 'Готовность к командировкам'], as_index=False)['ЗП (руб)'].median().sort_values(by='ЗП (руб)')

fig = px.bar(data, x='Готовность к переезду', color='Готовность к командировкам', y='ЗП (руб)', title='Зависимость медианной желаемой зарплаты от признаков «Готовность к переезду» и «Готовность к командировкам»')
fig.show()


Диаграмма подтверждает ожидание, что соискатели с большей гибкостью к переездам ожидают получать большую зарплату.

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
pivot = df.pivot_table(values='ЗП (руб)', index='Образование', columns='Возраст', aggfunc='median')
fig = px.imshow(pivot, title='Зависимость желаемой зарплаты от возраста и образования',
    color_continuous_scale=px.colors.diverging.Tealrose
)
fig.update_yaxes(autorange=True)
fig.show()

Сразу бросается в глаза
* Выбросы после 89 лет с нашим 100 летним соискателем и с соискателями моложе 20-ти лет имеющих высшие образование.
* Соискатели с высшим образование добиваются высоких зарплат быстрее
* Желаемая зарплата растёт с возрастом

4. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
data = df.copy()
data['Опыт работы (лет)'] = data['Опыт работы (месяц)'].apply(func=lambda z: np.nan if pd.isnull(z) else round(z / 12, 1))
fig = px.scatter(data, y='Опыт работы (лет)', x='Возраст', title='Зависимость Опыта работы от Возраста соискателей')
fig.add_trace(pg.Scatter(x=[0,100], y=[0,100], mode='lines'))
fig.update_layout(yaxis_range=[0,100], xaxis_range=[0,100], showlegend=False)
fig.show()

На диаграмме зависимости опыта работы от возраста стали очевидными несколько аномалий.
* 100 летний соискатель без опыта
* Несколько соискателей имеют опыта больше, чем их возраст
* И несколько соискателей почти на линии Опыт = Возраст, что означает начало опыта работы с дошкольного возраста.

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
filtered_df = df[df['Возраст'] < 76]
pivot = filtered_df.pivot_table(values='ЗП (руб)', index='Пол', columns='Возраст', aggfunc='median')
fig = px.imshow(pivot, title='Зависимость желаемой зарплаты от возраста и пола',
                color_continuous_scale=px.colors.diverging.Tealrose
                )
fig.update_yaxes(autorange=True)
fig.show()

filtered_df = filtered_df.copy()
filtered_df['Готовность к командировкам, %'] = filtered_df['Готовность к командировкам'].apply(func=lambda z: 100 if z else 0)
data = filtered_df.groupby('Возраст', as_index=False)['Готовность к командировкам, %'].mean()
fig = px.bar(data, x='Возраст', y='Готовность к командировкам, %',
    title='Показатель готовности к командировкам, % от возраста')
fig.show()


Первая диаграмма показывает зависимость зарплаты от поля и возраста.
* Как можно видеть в наш век мужчины все еще получают существенно больше чем женщины
* Женщины начинают работать с 17 лет, на пару лет позже чем мужчины и уходят на пенсию раньше

Вторая диаграммы показывает показатель готовности к командировкам, % от возраста
* Соискатели больше всего готовы к командировкам в возрасте 25-60 лет
* До 25 лет соискатели похоже не готовы к командировкам из-за школы/учебы
* А после 60 лет, ездить уже становится неудобно (здоровье уже не то и желание повидать новые места меньше)

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
cleaned_df = df.copy()
cleaned_df['Обновление резюме'] = pd.to_datetime(cleaned_df['Обновление резюме']).dt.date
display(cleaned_df.duplicated().sum())
cleaned_df.drop_duplicates(inplace=True)
display(cleaned_df.shape[0])

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
for col in cleaned_df.columns:
    nan_count = cleaned_df[col].isnull().sum()
    if nan_count > 0:
        print(f"'{col}' столбец имеет {nan_count} nan значений")

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
cleaned_df.dropna(
    axis='index',
    subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'],
    how='any',
    inplace=True
)
cleaned_df['Опыт работы (месяц)'].fillna(cleaned_df['Опыт работы (месяц)'].median(), inplace=True)

print(cleaned_df['Опыт работы (месяц)'].mean())

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
to_be_removed = cleaned_df[(cleaned_df['ЗП (руб)'] < 1_000) | (cleaned_df['ЗП (руб)'] > 1_000_000)].index
print(len(to_be_removed))
cleaned_df.drop(to_be_removed, axis='index', inplace=True)

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
to_be_removed = cleaned_df[cleaned_df['Опыт работы (месяц)'] > 12 * cleaned_df['Возраст']].index
print(len(to_be_removed))
cleaned_df.drop(to_be_removed, axis='index', inplace=True)

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**? В какую сторону **асимметрично логарифмическое распределение**? Найдите выбросы с помощью **метода z-отклонения** и удалите их из данных, используйте логарифмический масштаб, сделайте **послабление на 1 сигму (возьмите 4 сигмы)** в сторону, противоположную асимметрии. 

In [ ]:
def outliers_z_score_mod(mdf, feature, log_scale=False, left=3, right=3):
    if log_scale:
        z = np.log(mdf[feature])
    else:
        z = mdf[feature]
    mu = z.mean()
    sigma = z.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outer = mdf[(z < lower_bound) | (z > upper_bound)]
    inner = mdf[(z > lower_bound) & (z < upper_bound)]
    return outer, inner

outliers, cleaned = outliers_z_score_mod(cleaned_df, 'Возраст', log_scale=True, left=3, right=4)
print('outliers:', outliers.shape[0])
# outliers: 3
print('cleaned:', cleaned.shape[0])
# cleaned: 44482

fig = px.histogram(np.log(cleaned_df['Возраст']), title='Логарифмическое распределение по возрасту')
fig.update_layout(yaxis_title="Количество", xaxis_title='Возраст (логарифм)', showlegend=False)
fig.show()